# Importações Necessárias

In [1]:
import numpy as np;
import pandas as pd;

# Parâmetros de Entrada

In [2]:
baseDados = "BPressureNishiBook.dat";
baseRegras = "BPressureNishiBook.txt";

# Pré-Processamento Base Transacional

In [210]:
# lendo arquivo
mt = pd.read_table(baseDados, delim_whitespace=True, dtype="str", header=None);
# pre-processando
mtBinaria = pd.get_dummies(mt);
dados = mtBinaria.astype('int').to_numpy();
dadosNeg = abs(dados-1);

print(dados);

[[1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0]
 [1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0]
 [0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1]
 [1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1]
 [0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0]
 [1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1]
 [0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0]
 [0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0]
 [1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0]]


# Pré-processamento RAs

In [222]:
# lendo a base
regras = pd.read_table(baseRegras, sep="#", names=("AR", "sup", "cnf"));
# limpando os valores
regras["sup"] = regras["sup"].str.replace("SUP:", "");
regras["cnf"] = regras["cnf"].str.replace("CONF:", "");
regras["antc"], regras["cons"] =  regras["AR"].str.split("==>").str;
regras["cons"] = regras["cons"].str.split();
regras["antc"] = regras["antc"].str.split();

print(regras);

                       AR  sup                   cnf  antc                cons
0                6 ==> 1    5     0.5555555555555556   [6]                 [1]
1                1 ==> 6    5                    1.0   [1]                 [6]
2                4 ==> 2    3                    1.0   [4]                 [2]
3                2 ==> 4    3                    0.5   [2]                 [4]
4                5 ==> 2    3                    1.0   [5]                 [2]
...                   ...  ...                   ...   ...                 ...
1369  15 ==> 3 6 9 12 16    1     0.3333333333333333  [15]   [3, 6, 9, 12, 16]
1370  12 ==> 3 6 9 15 16    1                    0.1  [12]   [3, 6, 9, 15, 16]
1371  9 ==> 3 6 12 15 16    1    0.16666666666666666   [9]  [3, 6, 12, 15, 16]
1372  6 ==> 3 9 12 15 16    1     0.1111111111111111   [6]  [3, 9, 12, 15, 16]
1373  3 ==> 6 9 12 15 16    1                   0.25   [3]  [6, 9, 12, 15, 16]

[1374 rows x 5 columns]


# Suporte

In [224]:
def absSupp(itemset1, itemset2=[], negativo=False):
    itemset = itemset1 + itemset2;   
    # Se o suporte for negativo, usamos a base invertida.
    if negativo:
        base = dadosNeg;
    else:
        base = dados;
    # criando um vetor temporário (para não alterar a base de dados).
    tmp = np.copy(base[:,itemset[0]-1]);
    # multiplicando os vetores de cada item dentro do itemset
    for i in itemset:
        tmp *= base[:,i-1];   
    # Somando todos os elementos dos vetores
    return np.sum(tmp, axis=0);
    
    
def relSupp(itemset1, itemset2=[], negativo=False):
    return absSupp(itemset1, itemset2, negativo) / dados.shape[1];

      
print("12 6 ==> 13 #SUPP: " + str(absSupp([12, 6], [13]))) #5
print("12 17 14 ==> 2 #SUPP: " + str(absSupp([12, 17, 14], [2]))) #2

12 6 ==> 13 #SUPP: 5
12 17 14 ==> 2 #SUPP: 2


# Confiança

In [185]:
def conf(antc,consq):
    return abSuppRA(antc,consq)/abSupp(antc);


print(conf([12, 17, 14], [2]));# 1.0
print(conf([6, 1, 18], [13])); # .666
print(conf([6, 13], [1, 18])); # .2857142857142857

1.0
0.6666666666666666
0.2857142857142857


# Added Value

In [ ]:
def addedValue(antc, consq):
    return conf(antc, consq)-reSupp(consq);

# All-Confidence

In [ ]:
def allConf(itemset):
    maxSup = max([reSupp(i) for i in itemset])
    
    return reSupp(itemset)/maxSup

# Casual Support

In [ ]:
def casualSupp(ant, cons):
    return relSupp(ant, cons) + relSupp(ant, cons, negativo=True);

# Certainty Factor (CF)

In [225]:
def certFactor(ant, cons):
    return conf(ant,cons)-relSupp(cons)/relSupp(cons, negativo=True);